In [20]:
import numpy as np

In [42]:
def CE(y, v, epsilon=1e-15):
    y = np.clip(y, epsilon, 1 - epsilon)
    return -v*np.log(y)-(1-v)*np.log(1-y)


def sRel(x):
    return np.log(1+np.exp(np.clip(x, -709, 709)))

def sig(z:float):
    return np.clip(1/(1 + np.exp(-np.clip(z, -709, 709))), 1e-15, 1-1e-15)

def sRelDer(x):
    return np.exp(np.clip(x, -709, 709))/(1+np.exp(np.clip(x, -709, 709)))

def CEDer(y, v):
    epsilon = 1e-15
    y = np.clip(y, epsilon, 1 - epsilon)
    return (-v/y + (1-v)/(1-y))

def sigDer(x):
    return sig(x)*(1-sig(x))



In [ ]:
class NN:
    def __init__(self):
        self.W = []
        W1 = np.random.randn(2, 3) * 0.1
        W2 = np.random.randn(1, 3) * 0.1

        # self.X = np.ones(2)
        # self.v = 0
        self.W.append(W1)
        self.W.append(W2)


    # -------------------------- forward prop
    def forward_prop(self,X, v, W, Funcky):
        y = []
        z = []

        y0 = X

        y0 = np.insert(y0, 0, 1)

        y.append(y0)

        # print(y0)


        for i in range(len(W)):
            zi = W[i] @ y[-1]

            z.append(zi)

            # print(f"z{i+1} ---> {zi}")

            yoo = Funcky[i](zi)
            if(i < len(W) - 1):
                yoo = np.insert(yoo, 0, 1)

            y.append(yoo)

            # print(f"y{i+1} ---> {yoo}")


        Loss = CE(y[-1], v)
        # print("Loss --->",Loss)

        return (y,z,Loss)


    # --------------------------------------- Backward Propagation
    def backward_prob(self, W, y, z, v):
        yp = []
        zp = []
        dW = []

        y2p = CEDer(y[len(W)], v)
        z2p = sigDer(z[len(W)-1]) * y2p

        yp.append(y2p)
        zp.append(z2p)
        

        # -------------------------
        # print(f"y2p ---> {y2p} ------ z2p ---> {z2p}")
        for i in range(len(W)-1):
            yop = W[len(W)-1-i].T @ zp[-1] 
            yop = yop[1:]
            zop = sRelDer(z[len(W)-i-2]) * yop

            yp.append(yop)
            zp.append(zop)

            # print(f"y1p --> {yop} ---- z1p ---> {zop}")


        yp.reverse()
        zp.reverse()
        for i in range(len(W)):
            zpi = zp[i].reshape(-1,1)
            y[i] = y[i].reshape(-1,1)

        
            doW = zpi @ y[i].T 

            dW.append(doW)

        return dW



        

    def train(self, dataset, lr=0.1):
        delta = 10000000000000000000
        Loss = 0
        Old_Loss = 1000000
        total_loss = 0
        while delta > 0.0000001 : 
            
            Old_Loss = total_loss
            total_loss = 0
            for data in dataset :
                y,z,Loss = self.forward_prop(data[0], data[1], self.W, [sRel, sig])
                total_loss += Loss
                dW = self.backward_prob(self.W, y, z, data[1])
                for i in range(len(self.W)) : 
                    self.W[i] -= lr * dW[i]
                
            delta = abs(Old_Loss - total_loss)
        
        
    def test(self, dataset):
        for data in dataset :
            y,z,Loss = self.forward_prop(data[0], data[1], self.W, [sRel, sig])
            print(data[0],y[-1], Loss)


nur = NN()

dataset = [[[1,1],0], [[0,0],0], [[1,0],1], [[0,1],1]]

nur.train(dataset)

nur.test(dataset)




[1, 1] [0.00094682] [0.00094727]
[0, 0] [7.85625714e-05] [7.85656576e-05]
[1, 0] [0.99982549] [0.00017452]
[0, 1] [0.9998254] [0.00017462]


[1. 1. 1.]
z1 ---> [0.3 0.3]
y1 ---> [0.85435524 0.85435524]
z2 ---> [0.27087105]
y2 ---> [0.56730673]
Loss ---> [0.83772619]
y2p ---> [-2.31110599] ------ z2p ---> [-0.56730673]
y1p --> [-0.05673067 -0.05673067] ---- z1p ---> [-0.03258851 -0.03258851]
--------------dW----------------
 [array([[-0.03258851, -0.03258851, -0.03258851],
       [-0.03258851, -0.03258851, -0.03258851]]), array([[-0.56730673, -0.48468148, -0.48468148]])]